In [1]:
import sys
sys.path.insert(1, '/home/b3arjuden/crocket')

from datetime import datetime
from decimal import Decimal
import itertools
from math import floor
from numpy import exp, linspace, median, mean, std
from pprint import pprint
from os import environ
from os.path import join
from scipy.stats import mannwhitneyu, ttest_ind, variation


import matplotlib.pyplot as plt
%matplotlib inline

from crocket.sql.sql import Database
from crocket.utilities.credentials import get_credentials

In [2]:
# Environment variables

HOME_DIRECTORY_PATH = environ['HOME']
CREDENTIALS_FILE_PATH = join(HOME_DIRECTORY_PATH, '.credentials_unlocked.json')

HOSTNAME = 'localhost'
DATABASE_NAME = 'TEST23'

In [3]:
# Load SQL username and password
USERNAME, PASSCODE = get_credentials(CREDENTIALS_FILE_PATH)

# Create database connection
db = Database(hostname=HOSTNAME,
              username=USERNAME,
              password=PASSCODE,
              database_name=DATABASE_NAME)

In [4]:
tables = db.get_all_tables()

In [ ]:
def call(time, wprice, buyvolume, buyorder, 
         duration=None, 
         price_lag_time=None, 
         price_lag_duration=None,
         price_lag_threshold=None,
         volume_lag_duration=None,
         volume_lag_threshold=None,
         profit_percent=None, 
         stop_loss_percent=None, 
         stop_gain_percent=None, 
         max_hold_time=None, 
         wait_time=None):
    
    calls = []
    
    bought = False
    stop_gain = False
    maximize_gain = False
    
    for ii in range(duration+volume_lag_duration, len(buyvolume)):
        
        # Action if haven't bought coin
        if not bought:
            
            # No action if purchased within time of last buy
            try:
                if ii - calls[-1].get('start') < wait_time:
                    continue
            except IndexError:
                pass
                
            sample_volume_mean = mean(buyvolume[ii-duration:ii])
            volume_lag_median = median(buyvolume[ii-duration-volume_lag_duration:ii-duration])
            
            if sample_volume_mean > 0 and volume_lag_median < volume_lag_threshold:
                sample_price = float(wprice[ii-1])
                previous_price = float(mean(wprice[ii-duration-price_lag_time:ii-duration-price_lag_time+price_lag_duration]))

                if sample_volume_mean > 2 and \
                abs((sample_price - previous_price)/previous_price) < price_lag_threshold and \
                sum([1 if x > 1 else 0 for x in buyvolume[ii-duration:ii]]) >= 3:
                    
                    bought = True
                    calls.append({'start': ii-1,  # buy time offset by 1 minute, account for time to purchase
                                  'time': time[ii-1],
                                  'buy_price': float(wprice[ii-1]),
                                  'buy_volume': sample_volume_mean,
                                  'buy_volume_total': sum(buyvolume[ii-duration:ii])}) 

        # Action if have bought coin
        else:
            
            # Set stop loss after X minutes after buy
            if ii - calls[-1].get('start') > 0 and \
            wprice[ii] < (calls[-1].get('buy_price') * (1 - stop_loss_percent)):
                
                print(ii, 'Passed stop loss.')
                calls[-1]['stop'] = ii # sell price offset by 1 minute, account for time to sell
                calls[-1]['sell_price'] = float(wprice[ii])
                calls[-1]['profit'] = ((calls[-1].get('sell_price') - calls[-1].get('buy_price'))/calls[-1].get('buy_price')) * 0.9975**2
                
                bought = False
                stop_gain = False
                
                continue
            
            # Activate stop gain signal after passing threshold percentage
            if wprice[ii] > (calls[-1].get('buy_price') * (stop_gain_percent + 1)):
                stop_gain = True
            
            # Activate maximize gain signal after passing profit threshold
            if wprice[ii] > (calls[-1].get('buy_price') * (profit_percent + 1)):
                maximize_gain = True
            
            # Sell after hitting profit threshold followed by drop in price of X%
            if maximize_gain:
#                 if (wprice[ii] - wprice[ii-1])/wprice[ii-1] < -0.005 or \
#                 (wprice[ii] < wprice[ii-1] < wprice[ii-2]) or \
#                 wprice[ii] < (calls[-1].get('buy_price') * (profit_percent + 1)):
                calls[-1]['stop'] = ii # sell price offset by 1 minute, account for time to sell
                calls[-1]['sell_price'] = float(wprice[ii])
                calls[-1]['profit'] = ((calls[-1].get('sell_price') - calls[-1].get('buy_price'))/calls[-1].get('buy_price')) * 0.9975**2

                bought = False
                stop_gain = False
                maximize_gain = False

                continue
                
            # Sell after passing max hold time
            if not maximize_gain and \
            (ii - calls[-1].get('start')) > max_hold_time:
                

                calls[-1]['stop'] = ii # sell price offset by 1 minute, account for time to sell
                calls[-1]['sell_price'] = float(wprice[ii])
                calls[-1]['profit'] = ((calls[-1].get('sell_price') - calls[-1].get('buy_price'))/calls[-1].get('buy_price')) * 0.9975**2
                
                print('Passed target hold time. Time: {}, Buy: {}, Sell: {}, Profit: {}'.format(
                    str(calls[-1].get('time')), str(calls[-1].get('buy_price')), 
                    str(calls[-1].get('sell_price')), str(calls[-1].get('profit'))))
                
                bought = False
                stop_gain = False
            
            # Sell after detecting stop gain signal and price drop below stop gain price
            elif stop_gain and wprice[ii] < (calls[-1].get('buy_price') * (stop_gain_percent + 1)):
                
                #print(ii, 'Passed stop gain then below stop gain, exited.')
                calls[-1]['stop'] = ii # sell price offset by 1 minute, account for time to sell
                calls[-1]['sell_price'] = float(wprice[ii])
                calls[-1]['profit'] = ((calls[-1].get('sell_price') - calls[-1].get('buy_price'))/calls[-1].get('buy_price')) * 0.9975**2

                bought = False
                stop_gain = False
                maximize_gain = False
    
    return calls

In [7]:
def run_algorithm(data, status,
                  duration=3,
                  price_lag_time=30,
                  price_lag_duration=5,
                  price_lag_threshold=0.05,
                  volume_lag_duration=60,
                  volume_lag_threshold=30,
                  profit_percent=0.05,
                  stop_loss_percent=0.01,
                  stop_gain_percent=0.01,
                  max_hold_time=14400,
                  wait_time=14400,
                  digits=Decimal('1e-8')):

    time = data.get('datetime')
    buyvolume = data.get('buy_volume')
    sellvolume = data.get('sell_volume')
    wprice = data.get('wprice')

    current_time = time[-1]
    current_price = wprice[-1]

    last_buy_time_difference = (current_time - status.get('last_buy').get('start')).total_seconds()

    # Action if haven't bought coin
    if not status.get('bought'):

        # No action if purchased within time of last buy
        if last_buy_time_difference < wait_time:
            return status

        sample_volume_mean = mean(buyvolume[-duration:])
        volume_lag_total = sum(buyvolume[-(duration + volume_lag_duration):-duration])

        if sample_volume_mean > 0 and volume_lag_total < volume_lag_threshold:
            previous_price = Decimal(
                mean(wprice[-(duration + price_lag_time):-(duration + price_lag_time - price_lag_duration)])).quantize(digits)

            if sample_volume_mean > 2 and \
                    abs((current_price - previous_price) / previous_price) < price_lag_threshold and \
                    sum([1 if x > 1 else 0 for x in buyvolume[-duration:]]) >= 3:
                # TODO: MAKE API CALL TO BUY (wrap in try)
                status['bought'] = True
                status['current_buy'] = {'start': current_time,
                                         'buy_price': current_price,
                                         'buy_volume': sample_volume_mean,
                                         'last30_buyvolume': sum(buyvolume[-33:-3]),
                                         'last60_buyvolume': sum(buyvolume[-63:-3]),
                                         'last30_sellvolume': sum(sellvolume[-33:-3]),
                                         'last60_sellvolume': sum(sellvolume[-63:-3])}

    # Action if have bought coin
    else:
        current_buy = status.get('current_buy')
        current_buy_hold_time = (current_time - current_buy.get('start')).total_seconds()

        stop_gain_threshold = (current_buy.get('buy_price') * Decimal(stop_gain_percent + 1)).quantize(digits)

        # Activate stop gain signal after passing threshold percentage
        if current_price > stop_gain_threshold:
            status['stop_gain'] = True

        # Activate maximize gain signal after passing profit threshold
        if current_price > (current_buy.get('buy_price') * Decimal(profit_percent + 1)).quantize(digits):
            status['maximize_gain'] = True

        # Sell if hit stop loss
        # Sell after hitting profit threshold followed by drop in price of X%
        # Sell after passing max hold time
        # Sell after detecting stop gain signal and price drop below stop gain price
        if (current_price < (current_buy.get('buy_price') * Decimal(1 - stop_loss_percent)).quantize(digits)) or \
                status.get('maximize_gain') or \
                current_buy_hold_time > max_hold_time or \
                (status.get('stop_gain') and current_price < stop_gain_threshold):
            # TODO: MAKE API CALL TO SELL (wrap in try)
            status['current_buy']['stop'] = current_time

            buy_price = current_buy.get('buy_price')

            status['current_buy']['sell_price'] = current_price
            status['current_buy']['profit'] = (((current_price - buy_price) / buy_price) * Decimal(0.995)).quantize(digits)

            status['bought'] = False
            status['stop_gain'] = False
            status['maximize_gain'] = False

            status['last_buy'] = {'start': current_buy.get('start'),
                                  'buy_price': buy_price}

    return status

In [8]:
def benchmark(skip_list, params):
    
    markets = db.get_all_tables()
    
    running_data = {}
    status = {}
    all_calls = {}
    
    bought_time = datetime(2017, 11, 11, 11, 11).astimezone(tz=None)

    last_buy = {'start': bought_time,
                'buy_price': 0}
    
    total_profit = 0
    
    for market in markets:
        status[market] = {'bought': False,
                          'last_buy': last_buy,
                          'current_buy': {},
                          'stop_gain': False,
                          'maximize_gain': False}

        running_data[market] = {'datetime': [],
                        'wprice': [],
                        'buy_volume': [],
                        'sell_volume': []}
        
        all_calls[market] = []
        
    for market in markets:
        
        if market not in skip_list:
            data = db.select_query(market, '*')
            
            if data:
                time, price, wprice, volume, buyvolume, sellvolume, buyorder, sellorder = zip(*data)
                
                time = [x.astimezone(tz=None) for x in time]
                
                for ind in range(90, len(volume)):
                    running_data[market]['datetime'] = time[ind-90:ind]
                    running_data[market]['wprice'] = wprice[ind-90:ind]
                    running_data[market]['buy_volume'] = buyvolume[ind-90:ind]
                    running_data[market]['sell_volume'] = sellvolume[ind-90:ind]
                    
                    status[market] = run_algorithm(running_data.get(market), status.get(market))
                    
                    completed_buy = status.get(market).get('current_buy')
                    
                    if completed_buy.get('profit'):
                        
                        status[market]['current_buy'] = {}
                        all_calls[market].append(completed_buy)
                        
                        
                if all_calls.get(market):
                    profit = sum([x.get('profit') for x in all_calls.get(market) if 'profit' in x])
                    print('For {}, profit: {}'.format(market, str(profit)))
                    
                    total_profit += profit
    
    print('Total profit: {}'.format(total_profit))
    
    return all_calls

In [9]:
skip_list = ['BTC-BCC', 'BTC-ETH', 'BTC-LSK', 'BTC-NEO', 'BTC-XRP', 'BTC-LTC']


calls = benchmark([])

For BTC-ADA, profit: 0.01702909
For BTC-ADX, profit: -0.01052404
For BTC-AGRS, profit: -0.02936602
For BTC-ARDR, profit: 0.04307815
For BTC-ARK, profit: -0.01681531
For BTC-BCC, profit: -0.01319082
For BTC-BTG, profit: 0.06291433
For BTC-CFI, profit: 0.07372787
For BTC-CVC, profit: -0.02104739
For BTC-DASH, profit: -0.01953494
For BTC-EDG, profit: -0.00649000
For BTC-EMC2, profit: -0.05602240
For BTC-ETC, profit: -0.00787778
For BTC-GNT, profit: 0.00946686
For BTC-GRS, profit: -0.02765125
For BTC-IOP, profit: -0.01151492
For BTC-KMD, profit: -0.02162131
For BTC-LSK, profit: 0.04588792
For BTC-LTC, profit: -0.00297139
For BTC-MCO, profit: 0.05601823
For BTC-MER, profit: -0.03053668
For BTC-MONA, profit: -0.02187295
For BTC-MTL, profit: 0.00764914
For BTC-NAV, profit: 0.11832990
For BTC-NEO, profit: 0.00500587
For BTC-OK, profit: -0.01144556
For BTC-OMG, profit: -0.03753836
For BTC-PAY, profit: -0.00469971
For BTC-PIVX, profit: 0.00949732
For BTC-POT, profit: -0.02153783
For BTC-POWR, pr

In [ ]:
pos_calls = []
neg_calls = []

for k in calls.keys():
    
    for c in calls.get(k):
        
        if c.get('profit') > 0.05:
            
            pos_calls.append(c)
        
        else:
            
            neg_calls.append(c)

In [20]:
total_profit = []
all_calls = []

for k in calls.keys():
    
    for x in calls.get(k):
        
        if x.get('last60_buyvolume') < 25:
            x['market'] = k
            total_profit.append(x.get('profit'))
            all_calls.append(x)
        
print('Total profit: {}'.format(str(sum(total_profit))))
print('Number of calls: {}'.format(str(len(total_profit))))

Total profit: 0.26546595
Number of calls: 141


In [ ]:
total_profit = []
all_calls = []

for k in calls.keys():
    
    for x in calls.get(k):
        
        if x.get('last60_sellvolume') < 30 and x.get('last60_buyvolume') < 30:
            x['market'] = k
            total_profit.append(x.get('profit'))
            all_calls.append(x)
        
print('Total profit: {}'.format(str(sum(total_profit))))
print('Number of calls: {}'.format(str(len(total_profit))))

In [ ]:
[(x.get('market'), x.get('profit'), x.get('start')) for x in all_calls]

In [ ]:
max(x1 + x2)

In [ ]:
field = 'last60_sellvolume'
x1 = [float(x.get(field)) for x in pos_calls]
x2 = [float(x.get(field)) for x in neg_calls]

bins = linspace(0, 150, 100)

plt.hist(x1, bins, alpha=0.6, color='b', label='POS')
plt.hist(x2, bins, alpha=0.3, color='r', label='NEG')
plt.show()

In [ ]:
list(itertools.chain.from_iterable([[calls.get(k)] for k in calls]))

In [ ]:
for k in sorted(calls.keys()):
    
    if calls.get(k):
        print('{}: total profit: {}, loss: {}, calls: {}, calls @ profit threshold: {}'.format(
            k, str(sum([x.get('profit') for x in calls[k] if 'profit' in x])), str(sum([x.get('profit') for x in calls[k] if 'profit' in x and x.get('profit') < 0])), 
            len(calls[k]), str(len([x for x in calls[k] if 'profit' in x and x.get('profit') > 0.05]))))

In [ ]:
mv = {'loss': [],
      'gain': []}

tv = {'loss': [],
      'gain': []}

tp = [0 for x in range(50)]

for k in sorted(calls.keys()):
    
    for e in calls.get(k):
        
        if e.get('profit') <= 0.02:
            mv['loss'].append(float(e.get('buy_volume')))
            tv['loss'].append(float(e.get('buy_volume_total')))
        else:
            mv['gain'].append(float(e.get('buy_volume')))
            tv['gain'].append(float(e.get('buy_volume_total')))
            
        tmp = floor(float(e.get('buy_volume_total')))

        tp[tmp] += float(e.get('profit'))
            
        print('{}: profit: {}, mean_volume: {}, total_volume: {}'.format(
            k, str(e.get('profit')), str(e.get('buy_volume')), str(e.get('buy_volume_total'))))

In [ ]:
calls.get('BTC-TIX')

In [ ]:
for ii in range(17, 29):

    orders = list(itertools.chain.from_iterable([[x for x in calls.get(y) if 0 < (x.get('time')-datetime(2017, 11, ii, 20, 14)).total_seconds() < 86400] 
                  for y in calls]))
    
    print('11/{}: {} orders, profit: {}'.format(str(ii), str(len(orders)), str(sum([x.get('profit') for x in orders if x.get('profit')]))))
    
    #pprint(sorted([(x.get('time'), x.get('profit')) for x in orders]))

In [ ]:
for x in sorted(calls.keys()):
    
    pprint((x, calls.get(x)))

In [ ]:
data = db.select_query('BTC-BAY', '*')

time, price, wprice, volume, buyvolume, sellvolume, buyorder, sellorder = zip(*data)

calls = call(time, wprice, buyvolume, buyorder,
                             duration=params.get('duration'),
                             price_lag_time=params.get('price_lag_time'),
                             price_lag_duration=params.get('price_lag_duration'),
                             price_lag_threshold=params.get('price_lag_threshold'),
                             volume_lag_duration=params.get('volume_lag_duration'),
                             volume_lag_threshold=params.get('volume_lag_threshold'),
                             profit_percent=params.get('profit_percent'),
                             stop_loss_percent=params.get('stop_loss_percent'),
                             stop_gain_percent=params.get('stop_gain_percent'),
                             max_hold_time=params.get('max_hold_time'),
                             wait_time=params.get('wait_time'))

In [ ]:
calls

In [ ]:
for table in tables:
    
    data = db.select_query(table, '*')

    time, price, wprice, volume, buyvolume, sellvolume, buyorder, sellorder = zip(*data)
    
    ind = [i for i in range(duration, len(buyvolume)) if mean(buyvolume[i-duration:i]) > 2 
           and variation(buyvolume[i-duration:i]) < 1
           and all([1 if x > 1 else 0 for x in buyvolume[i-duration:i]])]
    
    if ind:
        print('Found {} hits for {}: {}'.format(str(len(ind)), table, ','.join(map(str, ind))))
        plt.plot(range(len(time)), wprice, markevery=ind, marker='o', markerfacecolor='r', markersize=10)
        plt.show()
        plt.plot(range(len(time)), buyvolume, markevery=ind, marker='o', markerfacecolor='r', markersize=10)
        plt.show()
        for i in ind:
            print(i, buyvolume[i-duration:i])
            print(i, time[i])
        input('NEXT')
    else:
        print('No hits found for {}'.format(table))